<a href="https://colab.research.google.com/github/steinruck/WANG-690-FALL-2020/blob/master/Assignment_06/Assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06

1. Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html

In [1]:
import pandas as pd
import html5lib

In [2]:
# web scraping with pandas
url = 'https://www.genealogybranches.com/censuscosts.html'
dfs = pd.read_html(url)

In [3]:
# I just wanted to know what it was 
type(dfs)

list

In [4]:
# dfs.head() won't work because it is a list

2. Find out how many HTML tables Pandas retrieves from the web page. Find out which dataframe contain the data and use it for further processing.

In [5]:
len(dfs) # two tables as a list of tables

2

In [6]:
# first table is at index 0 and contains the census data
dfs = pd.read_html(url)[0]
dfs.head() # to verify

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [7]:
len(dfs) # table length is 24 total rows - includes header

24

3. Display and explore the data (rows, columns, etc.) and determine the data quality (bad rows, bad columns, bad elements, null values, etc.)
4. Document the data quality issues using a Markdown cell so the reader understands what the problems are.

In [8]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       24 non-null     object
 1   1       24 non-null     object
 2   2       24 non-null     object
 3   3       24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes


In [9]:
# Not registering the column titles as column titles. instead using int  
dfs.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [10]:
dfs.shape # 24 rows and 4 columns

(24, 4)

In [11]:
dfs.head(1) # shows first line as what I want the header to be 

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person


In [12]:
# change first line in table to header
header = dfs.iloc[0] 
header

0                Census Year
1           Total Population
2                Census Cost
3    Average Cost Per Person
Name: 0, dtype: object

In [13]:
dfs = dfs.iloc[1:] # dataframe is anything below the first line (header)
dfs.head()

,0,1,2,3
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


In [14]:
header_list = list(header) # makes header columns into list
header_list

['Census Year', 'Total Population', 'Census Cost', 'Average Cost Per Person']

In [15]:
dfs.columns = header_list # aligns columns to header columns
dfs.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


In [16]:
dfs.describe

<bound method NDFrame.describe of    Census Year Total Population     Census Cost Average Cost Per Person
1         1790          3929214         $44,377              1.13 cents
2         1800          5308483         $66,109              1.24 cents
3         1810          7239881        $178,445              2.46 cents
4         1820          9633822        $208,526              2.16 cents
5         1830         12866020        $378,545              2.94 cents
6         1840         17069458        $833,371              4.88 cents
7         1850         23191876      $1,423,351              6.14 cents
8         1860         31443321      $1,969,377              6.26 cents
9         1870         38558371      $3,421,198              8.87 cents
10        1880         50155783      $5,790,678             11.54 cents
11        1890         62979766     $11,547,127             18.33 cents
12        1900         76303387     $11,854,000             15.54 cents
13        1910         9197226

Doesn't display calculations because there is no uniformity to the data. Need to remove $ and commas. Convert cents to dollars. Remove extra characters.

In [17]:
dfs.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23,23,23,23
unique,23,23,23,23
top,1910,7239881,"$127,934,000",$1.22
freq,1,1,1,1


5. Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")

### Test data section

In [18]:
df = pd.DataFrame({'Year' : ['1989*', '20*10'], 'Money': ['$10,000', '$14 billion'], 'Math': ['$1.84', '1.12 cents']})
df

,Year,Money,Math
0,1989*,"$10,000",$1.84
1,20*10,$14 billion,1.12 cents


In [19]:
# function to remove asterisk from 2010
def clean_year(year_str):
    return str(year_str.replace('*', ''))
df['Year'] = df['Year'].apply(clean_year)
df

,Year,Money,Math
0,1989,"$10,000",$1.84
1,2010,$14 billion,1.12 cents


In [20]:
# function to remove extra characters form test data
def clean_numbers(row):
    return float(row.replace('$', '').replace(',', '').replace(' billion', '000000000').replace(' cents', ''))

In [21]:
# runs function on specified columns
df['Money'] = df['Money'].apply(clean_numbers)
df['Math'] = df['Math'].apply(clean_numbers)
df

,Year,Money,Math
0,1989,1.000000e+04,1.84
1,2010,1.400000e+10,1.12


In [22]:
# divides the cents number by 100 to convert to dollars (I don't know what this warning is)
df['Math'].iloc[1] = df['Math'].iloc[1] / 100

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [23]:
df # not sure why it displays the numbers like this

,Year,Money,Math
0,1989,1.000000e+04,1.8400
1,2010,1.400000e+10,0.0112


### End Test Data

6. Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.

In [24]:
# function to remove asterisk from year column
def cleanse_year(census_year):
    return str(census_year.replace('*', ''))

In [25]:
# apply function to Census Year colummn and display
dfs['Census Year'] = dfs['Census Year'].apply(cleanse_year)
dfs

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents
6,1840,17069458,"$833,371",4.88 cents
7,1850,23191876,"$1,423,351",6.14 cents
8,1860,31443321,"$1,969,377",6.26 cents
9,1870,38558371,"$3,421,198",8.87 cents
10,1880,50155783,"$5,790,678",11.54 cents



7. Repeat this process for column "Total Population", "Census Cost", "Average Cost Per Person". (define, test, and apply the function)

In [26]:
# Function to remove unwanted characters from Census Cost column
def cleanse_cost(census_cost):
    return census_cost.replace(',', '').replace('$', '').replace(' Billion', '000000000').replace('.50', '5')

In [27]:
# Function to remove unwanted characters from Average Cost Per Person Column
def cleanse_per(per_cost):
    return float(per_cost.replace(',', '').replace('$', '').replace(' cents', ''))
 

In [28]:
# Apply funtion to Census Cost column and display
dfs['Census Cost'] = dfs['Census Cost'].apply(cleanse_cost)
dfs

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,44377,1.13 cents
2,1800,5308483,66109,1.24 cents
3,1810,7239881,178445,2.46 cents
4,1820,9633822,208526,2.16 cents
5,1830,12866020,378545,2.94 cents
6,1840,17069458,833371,4.88 cents
7,1850,23191876,1423351,6.14 cents
8,1860,31443321,1969377,6.26 cents
9,1870,38558371,3421198,8.87 cents
10,1880,50155783,5790678,11.54 cents


In [29]:
# Apply function to Average Cost Per Person column and display 
dfs['Average Cost Per Person'] = dfs['Average Cost Per Person'].apply(cleanse_per)
dfs

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,44377,1.13
2,1800,5308483,66109,1.24
3,1810,7239881,178445,2.46
4,1820,9633822,208526,2.16
5,1830,12866020,378545,2.94
6,1840,17069458,833371,4.88
7,1850,23191876,1423351,6.14
8,1860,31443321,1969377,6.26
9,1870,38558371,3421198,8.87
10,1880,50155783,5790678,11.54


In [30]:
 # divide cents by 100 to convert to dollars
 dfs['Average Cost Per Person'].iloc[0:18] = dfs['Average Cost Per Person'].iloc[0:18] / 100

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [31]:
dfs

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,44377,0.0113
2,1800,5308483,66109,0.0124
3,1810,7239881,178445,0.0246
4,1820,9633822,208526,0.0216
5,1830,12866020,378545,0.0294
6,1840,17069458,833371,0.0488
7,1850,23191876,1423351,0.0614
8,1860,31443321,1969377,0.0626
9,1870,38558371,3421198,0.0887
10,1880,50155783,5790678,0.1154


8. After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.

In [32]:
# save to file
dfs.to_csv('census_cost_cleansed.csv', index=False)

9. Use Pandas to read the saved cleansed file and explore to make sure it is clean.

In [33]:
# read file 
pd.read_csv("census_cost_cleansed.csv")

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,44377,0.0113
1,1800,5308483,66109,0.0124
2,1810,7239881,178445,0.0246
3,1820,9633822,208526,0.0216
4,1830,12866020,378545,0.0294
5,1840,17069458,833371,0.0488
6,1850,23191876,1423351,0.0614
7,1860,31443321,1969377,0.0626
8,1870,38558371,3421198,0.0887
9,1880,50155783,5790678,0.1154


10. Updload both the Jupyter notebook and the cleanse data file to GitHub and submit the GitHub link in BlackBoard.